In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 7.4 MB/s eta 0:00:00m eta 0:00:010:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.9/777.9 kB 4.9 MB/s eta 0:00:004.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 7.8 MB/s eta 0:00:007.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 6.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 5.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.5 MB/s eta 0:00:008.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 5.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 7.5 MB/s eta 0:00:0

In [4]:
import cv2
from ultralytics import YOLO

## Check for GPU availability

In [14]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA RTX A4000


In [15]:
!nvcc --version
print("\npytorch version:", torch.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0

pytorch version: 2.3.0


## Initialize device to GPU/CPU usage

In [18]:
device = None
if(torch.cuda.is_available()):
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("using", device, "device")

using cuda device


## Initialize the YOLOv9 model

In [19]:
# set to desired gpu number
# torch.cuda.set_device(0)

# specify use of gpu in model
model = YOLO('yolov9e-seg.pt')

# use gpu for model
model.to(device)
torch.cuda.synchronize()

## Initialize video source

In [12]:
# Define video source
vid_path = "./sources/two_then_one_ppl.mp4"
vid_cap = cv2.VideoCapture(vid_path)
vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)

# Check if the video was openened correctly
if not vid_cap.isOpened():
    print("Error opening video file.")
else:
    print("Video opened successfully!")
    print("Video Frame Rate:", str(vid_fps) + " fps")

Video opened successfully!
Video Frame Rate: 30.0 fps


## Define YOLO classes

In [10]:
PERSON = 0
BIKE = 1
CAR = 2
MOTORCYCLE = 3
BUS = 5

## Process video source

In [14]:
ped_times = {}

while vid_cap.isOpened():
	success, frame = vid_cap.read()
	if not success:
		break

	# Perform detection and tracking
	results = model.track(frame, persist=True, conf=0.2, classes=[PERSON, BIKE], iou=0.5, show=False, tracker="bytetrack.yaml")

	# Print frame
	frame_num = int(vid_cap.get(cv2.CAP_PROP_POS_FRAMES))

	# Get annotated frame and display it
	# annotated_frame = results[0].plot()
	# cv2.imshow('YOLOv9 Tracking', annotated_frame)
	for i, r in enumerate(results):
		for index, box in enumerate(r.boxes):
			# id of object
			print(box.id)
			print(box.cls)
			if(box.id and box.cls[0] != None):
				# print(box.id)
				# print(box.cls)
				tracker_id = int(box.id)
				tracker_cls = int(box.cls[0])
				print("class:", tracker_cls)
				print("id", tracker_id)

				obj_idx = "class_" + str(tracker_cls) + "_idx_" + str(tracker_id)
				if(not(obj_idx in ped_times)):
					ped_times[obj_idx] = frame_num / vid_fps
    
    
	print("frame:", frame_num)

torch.cuda.synchronize()
print(ped_times)

	# # Press 'q' to exit
	# if cv2.waitKey(1) & 0xFF == ord('q'):
	# 	break


0: 384x640 2 persons, 1052.0ms
Speed: 38.4ms preprocess, 1052.0ms inference, 106.6ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 1

0: 384x640 2 persons, 949.7ms
Speed: 1.9ms preprocess, 949.7ms inference, 25.6ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 2

0: 384x640 2 persons, 1057.2ms
Speed: 1.6ms preprocess, 1057.2ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 3

0: 384x640 2 persons, 1009.3ms
Speed: 1.6ms preprocess, 1009.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 4

0: 384x640 2 persons, 935.4ms
Speed: 1.8ms preprocess, 935.4ms inference, 18.0ms postprocess per image at shape (1, 3

## Process and Display video source

In [20]:
# Define video source
vid_path = "./sources/2024-05-10T08-45-36.mp4"
vid_cap = cv2.VideoCapture(vid_path)
vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)

# Check if the video was openened correctly
if not vid_cap.isOpened():
    print("Error opening video file.")
else:
    print("Video opened successfully!")
    print("Video Frame Rate:", str(vid_fps) + " fps")

# Specify use of gpu in model
model = YOLO('yolov9e-seg.pt')

# Use gpu for model
model.to(device)
torch.cuda.synchronize()

# Read and process the video
while vid_cap.isOpened():

    # Get the next captured frame
    success, frame = vid_cap.read()
    if not success:
        break

    # Perform detection and tracking on frame
    results = model.track(
        frame,
        persist=True,
        conf=0.2,
        iou=0.5,
        show=False,
        tracker="bytetrack.yaml",
        classes=[PERSON]
    )

    # Get annotated frame and display it
    annotated_frame = results[0].plot()
    cv2.imshow('YOLOv9 Tracking', annotated_frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
torch.cuda.synchronize()
vid_cap.release()
cv2.destroyAllWindows()

Video opened successfully!
Video Frame Rate: 30.0 fps

0: 384x640 (no detections), 20.2ms
Speed: 2.1ms preprocess, 20.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.5ms
Speed: 2.3ms preprocess, 21.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.2ms
Speed: 2.8ms preprocess, 21.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.1ms
Speed: 2.1ms preprocess, 20.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.9ms
Speed: 2.0ms preprocess, 19.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.9ms
Speed: 2.7ms preprocess, 20.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


QObject::moveToThread: Current thread (0x346f6730) is not the object's thread (0x3420fbd0).
Cannot move to target thread (0x346f6730)

QObject::moveToThread: Current thread (0x346f6730) is not the object's thread (0x3420fbd0).
Cannot move to target thread (0x346f6730)

QObject::moveToThread: Current thread (0x346f6730) is not the object's thread (0x3420fbd0).
Cannot move to target thread (0x346f6730)

QObject::moveToThread: Current thread (0x346f6730) is not the object's thread (0x3420fbd0).
Cannot move to target thread (0x346f6730)

QObject::moveToThread: Current thread (0x346f6730) is not the object's thread (0x3420fbd0).
Cannot move to target thread (0x346f6730)

QObject::moveToThread: Current thread (0x346f6730) is not the object's thread (0x3420fbd0).
Cannot move to target thread (0x346f6730)

QObject::moveToThread: Current thread (0x346f6730) is not the object's thread (0x3420fbd0).
Cannot move to target thread (0x346f6730)

QObject::moveToThread: Current thread (0x346f6730) is n


0: 384x640 (no detections), 20.5ms
Speed: 2.3ms preprocess, 20.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.1ms
Speed: 1.9ms preprocess, 20.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.8ms
Speed: 1.9ms preprocess, 19.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.2ms
Speed: 2.7ms preprocess, 20.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.3ms
Speed: 2.5ms preprocess, 20.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.9ms
Speed: 2.3ms preprocess, 19.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.5ms
Speed: 1.9ms preprocess, 19.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.1ms
Speed: 1.8ms preprocess, 20.1ms i


0: 384x640 (no detections), 21.1ms
Speed: 2.2ms preprocess, 21.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.6ms
Speed: 2.1ms preprocess, 20.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.8ms
Speed: 1.8ms preprocess, 20.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.5ms
Speed: 2.1ms preprocess, 20.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.7ms
Speed: 2.0ms preprocess, 20.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.0ms
Speed: 2.2ms preprocess, 21.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.4ms
Speed: 2.5ms preprocess, 20.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.7ms
Speed: 2.1ms preprocess, 20.7ms i

Speed: 2.3ms preprocess, 21.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.6ms
Speed: 2.0ms preprocess, 22.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 20.7ms
Speed: 2.1ms preprocess, 20.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.0ms
Speed: 2.2ms preprocess, 21.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 23.1ms
Speed: 2.1ms preprocess, 23.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.4ms
Speed: 2.0ms preprocess, 22.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.8ms
Speed: 2.2ms preprocess, 21.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.1ms
Speed: 2.4ms preprocess, 21.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person,

...

Speed: 2.1ms preprocess, 22.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.3ms
Speed: 2.0ms preprocess, 22.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.2ms
Speed: 2.0ms preprocess, 22.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.2ms
Speed: 2.1ms preprocess, 22.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.2ms
Speed: 2.1ms preprocess, 22.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.3ms
Speed: 2.1ms preprocess, 21.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.8ms
Speed: 2.2ms preprocess, 21.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person,


0: 384x640 1 person, 22.2ms
Speed: 2.2ms preprocess, 22.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.7ms
Speed: 2.1ms preprocess, 21.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.0ms
Speed: 2.1ms preprocess, 22.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.5ms
Speed: 2.1ms preprocess, 22.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.4ms
Speed: 2.1ms preprocess, 21.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.7ms
Speed: 2.0ms preprocess, 21.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.3ms
Speed: 2.2ms preprocess, 21.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 1.5ms postprocess per image at shape (1, 3, 38


0: 384x640 1 person, 22.7ms
Speed: 2.7ms preprocess, 22.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 23.1ms
Speed: 2.2ms preprocess, 23.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.8ms
Speed: 2.0ms preprocess, 21.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.9ms
Speed: 1.9ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.5ms
Speed: 2.2ms preprocess, 21.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.9ms
Speed: 2.0ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.1ms
Speed: 2.1ms preprocess, 22.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.5ms
Speed: 2.1ms preprocess, 21.5ms inference, 1.7ms postprocess per image at shape (1, 3, 38


0: 384x640 1 person, 21.8ms
Speed: 2.2ms preprocess, 21.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.9ms
Speed: 2.3ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.3ms
Speed: 2.0ms preprocess, 21.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.9ms
Speed: 2.4ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.8ms
Speed: 2.9ms preprocess, 21.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.4ms
Speed: 2.8ms preprocess, 22.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 20.9ms
Speed: 2.1ms preprocess, 20.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21.1ms
Speed: 2.2ms preprocess, 21.1ms inference, 1.5ms postprocess per image at shape (1, 3, 38

KeyboardInterrupt: 

## Process and Display video source

In [ ]:
# Define video source
vid_path = "./sources/2024-05-10T08-45-36.mp4"
vid_cap = cv2.VideoCapture(vid_path)
vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)

# Check if the video was openened correctly
if not vid_cap.isOpened():
    print("Error opening video file.")
else:
    print("Video opened successfully!")
    print("Video Frame Rate:", str(vid_fps) + " fps")

# Specify use of gpu in model
model = YOLO('yolov9e-seg.pt')

# Use gpu for model
model.to(device)
torch.cuda.synchronize()

# Read and process the video
while vid_cap.isOpened():

    # Get the next captured frame
    success, frame = vid_cap.read()
    if not success:
        break

    # Perform detection and tracking on frame
    results = model.track(
        frame,
        persist=True,
        conf=0.2,
        iou=0.5,
        show=False,
        tracker="bytetrack.yaml",
        classes=[PERSON]
    )

    # Get annotated frame and display it
    annotated_frame = results[0].plot()
    cv2.imshow('YOLOv9 Tracking', annotated_frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
torch.cuda.synchronize()
vid_cap.release()
cv2.destroyAllWindows()

In [22]:
torch.cuda.synchronize()
vid_cap.release()
cv2.destroyAllWindows()